# **Crypto market capitalization forecast based on S&P 500.**

## **Abstract**
   Abstract here. Give an executive summary of your project: goal, methods, results, conclusions. Usually no more than 200 words.


## **Introduction**

Here you have to explain the problem that you are solving. Explain why it is important, and what are the main challenges. Mention previous attempts (add papers as references) to solve it. Mainly focus on the techniques closely related to our approach. Briefly describe your approach and explain why it is promising for solving the addressed problem. Mention the dataset and the main results achieved.

In this section, you can add **text** and **figures**.

## **Methodology**
Describe the important steps you took to achieve your goal. Focus more on the most important steps (preprocessing, extra features, model aspects) that turned out to be important. Mention the original aspects of the project and state how they relate to existing work.

In this section, you can add **text** and **figures**. For instance, it is strongly suggested to add a picture of the best machine learning model that you implemented to solve your problem (and describe it).


### **Preprocessing**

The first step in our methodology involved preprocessing the raw data from 2 sources: Kaggle and CoinCodex. We will be using the Kaggle data for everything that is related to the S&P500, and CoinCodex for everything related crypto. For the cryptocurrency data, we focused on key features such as Date, Volume, and Marketcap. Similarly, for the S&P500 data, we retained relevant columns like Date, Open, High, Low, Close, Volume, and additional info regarding the fear index (VIX). The datasets were cleaned to handle missing values, if any, unwanted data and the Date columns were standardized to ensure compatibility for merging.

First Let's import the necessary libraries that we need for the project and define some constants!
Run the code below...


In [82]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [115]:
DATA_PATH = 'Data/'
STOCK_DATA_PATH_RAW = 'Data/StockData/RawData/'
STOCK_DATA_PATH_PROCESSED = 'Data/StockData/preProcessedData/'
CRYPTO_DATA_PATH_RAW = 'Data/CryptoData/RawData/'
CRYPTO_DATA_PATH_PROCESSED = 'Data/CryptoData/PreProcessedData/'
KAGGLE_DATA_PATH = 'Data/KaggleData/'
START_DATE = '2018-01-18'
END_DATE = '2025-04-04'

Once the libraries imported, we can now load the S&P500 data, and an example of crypto data to take a look at the first few rows along with some additional info by running the code below.

In [116]:
def load_data(filename: str, date_col: str, date_format: str) -> pd.DataFrame:
    """
    Loads a CSV file into a pandas DataFrame and parses the date column.

    Args:
        filename (str): Name of the CSV file.
        date_col (str): Name of the date column.
        date_format (str): Format of the date in the CSV.

    Returns:
        pd.DataFrame: Processed DataFrame with parsed dates.
    """
    filepath = os.path.join(STOCK_DATA_PATH_RAW, filename)
    df = pd.read_csv(filepath)
    df[date_col] = pd.to_datetime(df[date_col], format=date_format)
    return df

stock_df = load_data('S&P500_Historical_Data.csv', 'Date', '%Y-%m-%d')
vix_df = load_data('VIX_Historical_Data.csv', 'Date', '%m/%d/%Y')

for name, df in zip(["Stock", "VIX"], [stock_df, vix_df]):
    print(f"\n{name} Dataset ==> Min Date: {df['Date'].min()} / Max Date: {df['Date'].max()}")



Stock Dataset ==> Min Date: 2017-01-03 00:00:00 / Max Date: 2025-04-04 00:00:00

VIX Dataset ==> Min Date: 1990-01-02 00:00:00 / Max Date: 2025-04-04 00:00:00


In [117]:
stock_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2025-04-04,"5,074.08","5,292.14","5,292.14","5,069.90",NaN,-5.97%
1,2025-04-03,"5,396.52","5,492.74","5,499.53","5,390.83",NaN,-4.84%
2,2025-04-02,"5,670.97","5,580.76","5,695.31","5,571.48",NaN,0.67%
3,2025-04-01,"5,633.07","5,597.53","5,650.57","5,558.52",NaN,0.38%
4,2025-03-31,"5,611.85","5,527.91","5,627.56","5,488.73",NaN,0.55%


In [118]:
vix_df.head()

,Date,Open,High,Low,Close
0,1990-01-02,17.24,17.24,17.24,17.24
1,1990-01-03,18.19,18.19,18.19,18.19
2,1990-01-04,19.22,19.22,19.22,19.22
3,1990-01-05,20.11,20.11,20.11,20.11
4,1990-01-08,20.26,20.26,20.26,20.26


In [119]:
btc_df = pd.read_csv(os.path.join(CRYPTO_DATA_PATH_RAW, 'BTC.csv'))
btc_df.head()

,Start,End,Open,High,Low,Close,Volume,Market Cap
0,2025-04-06,2025-04-07,83533.45,83704.76,77296.39,78310.34,2.974769e+10,1.626852e+12
1,2025-04-05,2025-04-06,83769.12,84219.70,82384.97,83582.03,5.424886e+10,1.654110e+12
2,2025-04-04,2025-04-05,83259.08,84676.27,81767.53,83879.86,6.263226e+10,1.654911e+12
3,2025-04-03,2025-04-04,82259.03,83781.70,81307.75,83199.95,7.766843e+10,1.643472e+12
4,2025-04-02,2025-04-03,85170.68,87898.01,82487.40,82548.31,5.237611e+10,1.688190e+12


#### **Crypto Data**

Great, let's now tackle our raw crypto data.  
After some research and many hours of trying to find the best balance between variety of cryptos and the amount of data that can be used, I have decided to include 14 cryptos from the top 100 where the data stretches from 2018 to 2025.  

Let us take a look at an example of crypto data that we have:

**Note**: We have a CSV for every crypto. (14 CSVs total)

Let us standardize our timeframe now. From all the CSVs we have, we can see that our range should be from 2018-1-18 to 2025-04-04 to match the maximum date of the stock market data and the minimum date of the crypto CSVs.

In [120]:
date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
processed_dfs = {}  # Dictionary to store processed DataFrames (key: symbol, value: DataFrame)

for filename in os.listdir(CRYPTO_DATA_PATH_RAW):
    try:
        raw_path = os.path.join(CRYPTO_DATA_PATH_RAW, filename)
        df = pd.read_csv(raw_path)
        df['Start'] = pd.to_datetime(df['Start'])
        df = df[df['Start'].isin(date_range)]
        df = df.sort_values('Start')
        
        symbol = filename.split('.')[0]  # Extract symbol (e.g., 'BTC' from 'BTC.csv')
        processed_dfs[symbol] = df # Store processed DataFrame in the dictionary
        
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")

print("All files processed!")

All files processed!


Let us now check for some missing values in the most interesting features!

In [121]:
# Check for missing/invalid values (0, -1, or NaN) in key columns
for symbol, df in processed_dfs.items():
    invalid_volume = ((df['Volume'] == 0) | (df['Volume'] == -1) | (df['Volume'].isna())).sum()
    invalid_market_cap = ((df['Market Cap'] == 0) | (df['Market Cap'] == -1) | (df['Market Cap'].isna())).sum()
    
    if invalid_volume > 0 or invalid_market_cap > 0:
        print(f"\n{symbol}:")
        print(f"  ==> Missing/Invalid 'Volume' values: {invalid_volume}")
        print(f"  ==> Missing/Invalid 'Market Cap' values: {invalid_market_cap}")

print("\nCheck complete!")


BNB:
  ==> Missing/Invalid 'Volume' values: 0
  ==> Missing/Invalid 'Market Cap' values: 133

EOS:
  ==> Missing/Invalid 'Volume' values: 0
  ==> Missing/Invalid 'Market Cap' values: 133

Check complete!


After inspecting the dataframes of each token, we can see that there are some data points of market cap data missing in BNB and EOS.  
We will use data from kaggle to fill that in.  

In [122]:
def update_market_caps(processed_dfs, all_crypto_file):
    all_crypto_df = pd.read_csv(all_crypto_file)
    
    for symbol, crypto_df in processed_dfs.items():
        try:
            crypto_df['Start'] = pd.to_datetime(crypto_df['Start'])
            symbol_data = all_crypto_df[all_crypto_df['Symbol'] == symbol].copy()
            symbol_data['Date'] = pd.to_datetime(symbol_data['Date'], format='%d-%m-%Y %H:%M')
            
            # Normalize dates for comparison
            crypto_df['Start_date'] = crypto_df['Start'].dt.normalize()
            symbol_data['Date_date'] = symbol_data['Date'].dt.normalize()
            
            # Map market caps by date
            market_cap_dict = dict(zip(symbol_data['Date_date'], symbol_data['Marketcap']))
            
            # Update missing market caps
            updated_count = 0
            for index, row in crypto_df.iterrows():
                if row['Market Cap'] in [0.0, -1.0]:
                    start_date = row['Start_date']
                    if start_date in market_cap_dict:
                        crypto_df.at[index, 'Market Cap'] = market_cap_dict[start_date]
                        updated_count += 1
            
            crypto_df.drop(columns=['Start_date'], inplace=True)
            
            if updated_count > 0:
                print(f"{symbol}: Updated {updated_count} market cap values")
                
        except Exception as e:
            print(f"Error processing {symbol}: {str(e)}")

update_market_caps(processed_dfs, KAGGLE_DATA_PATH + 'All_Crypto.csv')
print("\nMarket caps updated!")

BNB: Updated 133 market cap values
EOS: Updated 133 market cap values

Market caps updated!


Nice, now our crypto data is complete within the time range of 2018 to 2025.  
Let us now work on dropping the unwated features and making the merged crypto dataset.

In [123]:
# Merge all DataFrames and add 'symbol' column
merged_df = pd.concat(
    [df.assign(symbol=symbol) for symbol, df in processed_dfs.items()],
    ignore_index=True
)

# Drop unwanted columns and rename
merged_df.drop(columns=['End', 'Low', 'High', 'Open', 'Close'], inplace=True)
merged_df.rename(columns={'Start': 'Date', 'Market Cap': 'MarketCap'}, inplace=True)

# Save the final merged file
output_file = 'Crypto.csv'
merged_df.to_csv(os.path.join(CRYPTO_DATA_PATH_PROCESSED, output_file), index=False)
print(f"Merged data saved to {output_file}")

Merged data saved to Crypto.csv


Let us now aggregate the data to have the daily total Market Cap and Voume.

In [124]:
# Aggregate the data by date so that each date has the sum of market caps and volumes for all cryptocurrencies
def aggregate_crypto_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates crypto data by date, summing 'Volume' and 'MarketCap',
    and renames the columns for clarity.
    
    Args:
        df (pd.DataFrame): The input DataFrame with 'Date', 'Volume', and 'MarketCap' columns.
    
    Returns:
        pd.DataFrame: Aggregated DataFrame with 'Date', 'Crypto_Volume', and 'Crypto_Market_Cap'.
    """
    aggregated_df = (
        df.groupby('Date', as_index=False)
        .agg({'Volume': 'sum', 'MarketCap': 'sum'})
        .rename(columns={'Volume': 'Crypto_Volume', 'MarketCap': 'Crypto_Market_Cap'})
    )

    return aggregated_df

In [125]:
crypto_df = aggregate_crypto_data(merged_df)
crypto_df.to_csv(os.path.join(CRYPTO_DATA_PATH_PROCESSED, 'Aggregated_Crypto.csv'), index=False)
print("Aggregated data saved to Aggregated_Crypto.csv")

Aggregated data saved to Aggregated_Crypto.csv


#### **S&P500 Data**

Let us now process the stock market data.  
We need to keep in mind that the stock market closes on the weekends. Thus, for the sake of this project, we will assume that the last available price (Friday’s) carries over to Saturday and Sunday since stock prices don’t change on weekends.  
We will use Forward fill to accomplish this.  
This will keep the dataset aligned with the crypto data. Also, it reflects the reality that stock prices remain unchanged on weekends.
Same thing will be done to the VIX data frame as well.

Let's define some helper functions...

In [126]:
def handle_closed_weekends(df: pd.DataFrame, start_date: str, end_date: str, date_col: str = 'Date') -> pd.DataFrame:
    """
    Processes S&P 500 data to:
    1. Filter date range
    2. Expand to include weekends
    3. Forward-fill price columns
    4. Set 'Change_%' to 0.0 on weekends
    5. Ensure proper datetime format
    
    Args:
        df: DataFrame with market data
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        date_col: Name of date column
    
    Returns:
        Processed DataFrame with continuous daily data
    """
    df[date_col] = pd.to_datetime(df[date_col])
    filtered_df = df[(df[date_col] >= start_date) & (df[date_col] <= end_date)].copy()
    
    full_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    if 'S&P500_Change_%' in filtered_df.columns:
        filtered_df['S&P500_Change_%'] = (
            filtered_df['S&P500_Change_%']
            .astype(str)
            .str.replace('%', '')
            .replace('', '0')
            .astype(float)
        )
    
    reindexed_df = (
        filtered_df
        .set_index(date_col)
        .reindex(full_dates)
    )
    
    price_cols = [col for col in reindexed_df.columns if col != 'Change_%']
    
    reindexed_df[price_cols] = reindexed_df[price_cols].ffill()
    
    if 'S&P500_Change_%' in reindexed_df.columns:
        reindexed_df['S&P500_Change_%'] = reindexed_df['S&P500_Change_%'].fillna(0.0)
    
    result = reindexed_df.reset_index().rename(columns={'index': date_col})
    
    result[date_col] = pd.to_datetime(result[date_col])
    
    return result

def preprocess_sp500(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    df = df.drop(columns=['Vol.'], errors='ignore')
    df['Open'] = df['Open'].str.replace(',', '').astype(float)
    df['High'] = df['High'].str.replace(',', '').astype(float)
    df['Low'] = df['Low'].str.replace(',', '').astype(float)
    df['Price'] = df['Price'].str.replace(',', '').astype(float)
    df = df.rename(columns={
            'Open': 'S&P500_Open',
            'High': 'S&P500_High',
            'Low': 'S&P500_Low',
            'Price': 'S&P500_Close',
            'Change %': 'S&P500_Change_%'
    })
    
    if not pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Date'] = pd.to_datetime(df['Date'])
    
    processed = handle_closed_weekends(
        df=df,
        start_date=start_date,
        end_date=end_date,
        date_col='Date'
    )
    
    return processed

def preprocess_vix(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.rename(columns={
            'Open': 'VIX_Open',
            'High': 'VIX_High',
            'Low': 'VIX_Low',
            'Close': 'VIX_Close'
        })
    
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    df = (
        df.set_index('Date')
        .reindex(date_range)
        .ffill()
        .reset_index()
        .rename(columns={'index': 'Date'})
    )
    
    return df

In [127]:
# Process both datasets
stock_df = preprocess_sp500(stock_df, START_DATE, END_DATE)
vix_df = preprocess_vix(vix_df, START_DATE, END_DATE)

# Print results
print(f"S&P500 dataset: {stock_df['Date'].min()} to {stock_df['Date'].max()} "
      f"with {len(stock_df)} rows")
print(f"Columns: {stock_df.columns.tolist()}\n")

print(f"VIX dataset: {vix_df['Date'].min()} to {vix_df['Date'].max()} "
      f"with {len(vix_df)} rows")
print(f"Columns: {vix_df.columns.tolist()}")

S&P500 dataset: 2018-01-18 00:00:00 to 2025-04-04 00:00:00 with 2634 rows
Columns: ['Date', 'S&P500_Close', 'S&P500_Open', 'S&P500_High', 'S&P500_Low', 'S&P500_Change_%']

VIX dataset: 2018-01-18 00:00:00 to 2025-04-04 00:00:00 with 2634 rows
Columns: ['Date', 'VIX_Open', 'VIX_High', 'VIX_Low', 'VIX_Close']


In [128]:
# Save the processed stock and VIX data
stock_df.to_csv(os.path.join(STOCK_DATA_PATH_PROCESSED, 'S&P500.csv'), index=False)
vix_df.to_csv(os.path.join(STOCK_DATA_PATH_PROCESSED, 'VIX.csv'), index=False)
print("Processed stock and VIX data saved!")

Processed stock and VIX data saved!


#### **Merging and Unification of the datasets**

Perfect, now that we have cleaned the data, we still have to unify our datasets into one before starting to play with models.  

In [133]:
temp_df = pd.merge(stock_df, vix_df, on='Date', how='inner')
main_df = pd.merge(temp_df, crypto_df, on='Date', how='inner')
main_df.to_csv(os.path.join(DATA_PATH, 'MainData.csv'), index=False)
main_df.head()

,Date,S&P500_Close,S&P500_Open,S&P500_High,S&P500_Low,S&P500_Change_%,VIX_Open,VIX_High,VIX_Low,VIX_Close,Crypto_Volume,Crypto_Market_Cap
0,2018-01-18,2798.0,2802.4,2805.8,2792.6,-0.16,12.01,12.40,11.62,12.22,3.397555e+10,4.182316e+11
1,2018-01-19,2810.3,2802.6,2810.3,2798.1,0.44,12.30,12.33,11.18,11.27,2.290271e+10,4.241106e+11
2,2018-01-20,2810.3,2802.6,2810.3,2798.1,0.44,12.30,12.33,11.18,11.27,1.770756e+10,4.568364e+11
3,2018-01-21,2810.3,2802.6,2810.3,2798.1,0.44,12.30,12.33,11.18,11.27,1.591606e+10,4.235859e+11
4,2018-01-22,2833.0,2809.2,2833.0,2808.1,0.81,11.59,11.62,10.84,11.03,1.512720e+10,4.032085e+11


#### **Feature Engineering**

Now that we have our main dataset, we can add meaningful features to it that will probably help with the training of the models.  
Let's define some helper functions.

In [134]:
# Help model to learn seasonal or cyclic patterns
def add_seasonal_features(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    """
    Adds seasonal features to the DataFrame based on the date column.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        date_col (str): The name of the date column.
    
    Returns:
        pd.DataFrame: DataFrame with added seasonal features.
    """
    df['Year'] = df[date_col].dt.year
    df['Month'] = df[date_col].dt.month
    df['Day'] = df[date_col].dt.day
    df['DayOfWeek'] = df[date_col].dt.dayofweek
    df['IsWeekend'] = df['DayOfWeek'] >= 5
    
    return df

# Add rolling statistics
def add_rolling_statistics(df: pd.DataFrame, window: int = 7) -> pd.DataFrame:
    """
    Adds rolling statistics to the DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        window (int): The rolling window size.
    
    Returns:
        pd.DataFrame: DataFrame with added rolling statistics.
    """
    df['Market_Cap_MA'] = df['Crypto_Market_Cap'].rolling(window=window).mean()
    df['Market_Cap_STD'] = df['Crypto_Market_Cap'].rolling(window=window).std()
    
    return df

# Add lag features
def add_lag_features(df: pd.DataFrame, lag: int = 1) -> pd.DataFrame:
    """
    Adds lag features to the DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        lag (int): The lag period.
    
    Returns:
        pd.DataFrame: DataFrame with added lag features.
    """
    df['Lagged_Market_Cap'] = df['Crypto_Market_Cap'].shift(lag)
    df['Lagged_VIX_Close'] = df['VIX_Close'].shift(lag)
    df['Lagged_S&P500_Close'] = df['S&P500_Close'].shift(lag)
    
    return df

# Add crypto percent change
def add_crypto_percent_change(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds percentage change for the Crypto_Market_Cap column.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
    
    Returns:
        pd.DataFrame: DataFrame with added percentage change feature.
    """
    df['Crypto_Market_Cap_%'] = df['Crypto_Market_Cap'].pct_change() * 100
    df['Crypto_Volume_Change_%'] = df['Crypto_Volume'].pct_change() * 100

    return df

In [135]:
# Add all features to the DataFrame
main_df = add_seasonal_features(main_df, 'Date')
main_df = add_rolling_statistics(main_df, window=7)
main_df = add_lag_features(main_df, lag=1)
main_df = add_crypto_percent_change(main_df)

main_df.to_csv(os.path.join(DATA_PATH, 'MainData_FeatureEngineering.csv'), index=False)
main_df.head()

,Date,S&P500_Close,S&P500_Open,S&P500_High,S&P500_Low,S&P500_Change_%,VIX_Open,VIX_High,VIX_Low,VIX_Close,...,Day,DayOfWeek,IsWeekend,Market_Cap_MA,Market_Cap_STD,Lagged_Market_Cap,Lagged_VIX_Close,Lagged_S&P500_Close,Crypto_Market_Cap_%,Crypto_Volume_Change_%
0,2018-01-18,2798.0,2802.4,2805.8,2792.6,-0.16,12.01,12.40,11.62,12.22,...,18,3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-19,2810.3,2802.6,2810.3,2798.1,0.44,12.30,12.33,11.18,11.27,...,19,4,False,NaN,NaN,4.182316e+11,12.22,2798.0,1.405692,-32.590615
2,2018-01-20,2810.3,2802.6,2810.3,2798.1,0.44,12.30,12.33,11.18,11.27,...,20,5,True,NaN,NaN,4.241106e+11,11.27,2810.3,7.716345,-22.683548
3,2018-01-21,2810.3,2802.6,2810.3,2798.1,0.44,12.30,12.33,11.18,11.27,...,21,6,True,NaN,NaN,4.568364e+11,11.27,2810.3,-7.278429,-10.117150
4,2018-01-22,2833.0,2809.2,2833.0,2808.1,0.81,11.59,11.62,10.84,11.03,...,22,0,False,NaN,NaN,4.235859e+11,11.27,2810.3,-4.810684,-4.956410


### **Model Training**